# 한국복지패널데이터

* <https://www.koweps.re.kr/>
* 주관: 한국보건사회연구원
* 목적: 국내 가구의 경제활동을 연구하여 정책 지원에 반영할 목적으로 조사
* 데이터 전처리 필요 -> 데이터 전처리와 시각화를 연습해봅시다
* source: [데이터 시각화 with 파이썬](http://book.naver.com/bookdb/book_detail.naver?bid=18123838)


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rc('font',family='NanumGothic')
mpl.rcParams['axes.unicode_minus']=False

In [ ]:
dpath = "/content/drive/MyDrive/colab_data/"

## 데이터 읽기

* `welfare_python.csv`

variable | detail | code
:--- | :--- | :--- 
gender | 성별 | 1=남, 2=여
birth | 태어난 연도 | YYYY
marriage | 혼인상태 | 0=비해당(18세미만), 1=배우자있음, 2=사별, 3=이혼, 4=별거, 5=미혼(18세이상, 미혼모포함), 6=기타(사망 등)
religion | 종교 유무 | 1=있음, 2=없음
code_job | 직업 코드 | float64(xxx) (`welfare_job.csv` 참고)
income | 소득(월평균 임금) | float64(xxx)
code_region | 7개 권역별 지역구분 | 1=서울, 2=수도권(인천/경기), 3=부산/경남/울산, 4=대구/경북, 5=대전/충남, 6=강원/충북, 7=광주/전남/전북/제주



In [ ]:
df = pd.read_csv(dpath + "welfare_python.csv")
print(df.head())
print(df.info())


In [ ]:
# unicode error 뜨는 경우 encoding="CP949" 옵션 추가
ref = pd.read_csv(dpath+'welfare_job.csv',encoding='CP949')
ref.head()

## 데이터 전처리

1. gender : 1이면 '남성', 2이면 '여성'으로 변경하여 gender2로 코딩
2. marriage : 1이면 '결혼', 3이면 '이혼', 나머지는 '그외'로 변경하여 marriage2로 코딩
3. code_job : `welfare_job.csv'에서 한글로 된 직업이름정보(job)를 추출하여 병합하여 job으로 코딩
4. code_region : 7개 권역별 지역이름을 문자열로 변경하여 region으로 변경
5. age : 현재 연도에서 태어난 연도를 빼서 신규컬럼 생성
6. religion : 1이면 '종교있음', 2이면 '무교'로 religion2로 표기
7. income : 소득이 없는 결측치의 경우 다른 월급들의 평균으로 대체 income2로 저장
8. agegroup : 연령(age)이 30세 미만이면 '청년', 30세 이상이면서 60세 미만은 '중년', 60세 이상이면 '노년'으로 신규컬럼 코딩 후 범주형 변수로 변경하기

In [ ]:
# 1. gender : 1이면 '남성', 2이면 '여성'으로 gender2로 코딩 (loc메소드 적용)
df.loc[df.gender==1,'gender2']='male'
df.loc[df.gender==2,'gender2']='female'
df.head()

In [ ]:
# 1. gender : 1이면 '남성', 2이면 '여성'으로 gender2로 코딩 (dictionary and for문 이용)
dic = {1:'male',2:'female'}
df['gender2'] = [dic[i] for i in df.gender]
df.head()

In [ ]:
# 2. marriage : 1이면 '결혼', 3이면 '이혼', 나머지는 '그외'라는 문자열로 처리하여 marriage2로 저장 (lambda함수 적용해보기)
def recoding(x):
  if x == 1:
    return 'married'
  elif x == 3:
    return 'divorced'
  else:
    return 'etc'

df['marriage2'] = df.marriage.apply(lambda x : recoding(x))
#df['marriage2'] = df.marriage.apply(recoding)
df.head()

In [ ]:
# 3. code_job : `welfare_job.csv'에서 한글로 된 직업이름정보(job)를 추출하여 병합 (merge이용)
df1 = pd.merge(df,ref,on='code_job')
print(df1.shape)
df.shape

In [ ]:
# 4. code_region : 7개 권역별 지역이름을 문자열로 표기하여 region에 저장 (함수 적용)
# 1=서울, 2=인천/경기, 3=부산/경남/울산, 4=대구/경북, 5=대전/충남, 6=강원/충북, 7=광주/전남/전북/제주
df = df1
print(df.code_region.value_counts())

def recoding(x):
  if x==1:
    return '서울'
  elif x==2:
    return '인천/경기'
  elif x==3:
    return '부산/경남/울산'
  elif x==4:
    return '대구/경북'
  elif x==5:
    return '대전/충남'
  elif x==6:
    return '강원/충북'
  else:
    return '광주/전남/전북/제주'

df['region'] = df.code_region.apply(recoding)
df.head()

In [ ]:
# 4. code_region : 7개 권역별 지역이름을 문자열로 표기하여 region에 저장 (dictionary 적용)
# 1=서울, 2=인천/경기, 3=부산/경남/울산, 4=대구/경북, 5=대전/충남, 6=강원/충북, 7=광주/전남/전북/제주
dic = {1:'서울',2:'인천/경기',3:'부산/경남/울산',4:'대구/경북',5:'대전/충남',6:'강원/충북',7:'광주/전남/전북/제주'}
df['region'] = [dic[i] for i in df.code_region]
df.head()

In [ ]:
# 5. age : 현재 연도에서 태어난 연도를 빼서 신규컬럼 생성
df['age'] = 2021 - df['birth']
df.head()

In [ ]:
# 5. age : 현재 연도에서 태어난 연도를 빼서 신규컬럼 생성 (현재 연도를 함수로 가져와보기)
# 구글에서 'year in python' 검색
import datetime
dt_now = datetime.datetime.now()
dt_now.year
df['age'] = dt_now.year - df.birth

In [ ]:
# 6. religion : 1이면 '종교있음', 2이면 '무교'로 표기하여 religion2로 코딩 (if else 구문 이용)
df['religion2'] = ['유' if i ==1 else '무' for i in df.religion]
print(df.head())
df.religion2.value_counts()

In [ ]:
# 7. income : 소득이 없는 결측치의 경우 다른 월급들의 평균으로 대체하여 income2로 코딩
print(sum(df.income.isna()))

mean = df.income.mean()
df['income2'] = df.income.fillna(mean)
print(df.head())
print(sum(df.income2.isna()))

In [ ]:
# 8. agegroup : 연령(age)이 30세 미만이면 '청년', 30세 이상이면서 60세 미만은 '중년', 60세 이상이면 '노년'으로 신규컬럼 코딩 (함수 이용)
def recoding(x):
  if x<30:
    return '청년'
  elif x<60:
    return '중년'
  else:
    return '노년'

df['agegroup'] = df.age.apply(recoding)
print(df.agegroup.value_counts())
df.head()

In [ ]:
# 8. 범주형 변수로 순서 카테고리 지정하기
df.agegroup = pd.Categorical(df.agegroup, categories=['청년','중년','노년'])
df.agegroup

In [ ]:
# 9. 다음 변수만 모아서 한글변수명을 붙이고 df2에 저장
# age=연령, agegroup=연령대, birth=생년, gender2=성별, marriage2=결혼상태, religion2=종교유무, 
# region=지역, job=직업, income2=월소득
df2 = df[['age','agegroup','birth','gender2','marriage2','religion2','region','job','income2']]
df2.head()

In [ ]:
print(df2.columns)
dic = {'age':'연령','agegroup':'연령대','birth':'생년','gender2':'성별','marriage2':'결혼상태','religion2':'종교유무','region':'지역','job':'직업','income2':'월소득'}
df2.rename(columns=dic,inplace=True)
df2.head()

In [ ]:
# 위치가 변경될 수 있으니 dictionary 이용


In [ ]:
# df2를 /contents/drive/MyDrive/data/output/welfare_clean.csv로 저장
df2.to_csv(dpath+"welfare_clean.csv",index=False)

In [ ]:
# welfare_clean.csv를 불러와서 잘 저장되었는지 확인해보기
df = pd.read_csv(dpath+'welfare_clean.csv')
df.head()

In [ ]:
# df2를 /contents/drive/MyDrive/data/output/welfare_clean.csv로 저장 (index 없이 저장하기)


# welfare_clean.csv를 불러와서 잘 저장되었는지 확인해보기


## 데이터 시각화

### 결혼상태와 종교유무의 분포

In [ ]:
# 결혼상태별 빈도 확인
df['결혼상태'].value_counts()

In [ ]:
# 빈도막대그래프 그리기
sns.catplot(kind='count',x='결혼상태',data=df);

In [ ]:
# 결혼상태 순서를 정해주기
df['결혼상태'] = pd.Categorical(df['결혼상태'],categories=['married','divorced','etc'])

In [ ]:
sns.catplot(kind='count',x='결혼상태',data=df);

In [ ]:
# countplot()의 order 옵션 사용해서 이혼, 결혼, 그외 순으로 그림 그려보기
sns.catplot(kind='count',x='결혼상태',data=df,order=['divorced','married','etc']);

In [ ]:
# 결혼상태에 따라 종교유무를 나누어 표기
# 막대 색상을 팔레트 "Set2" 이용
sns.catplot(kind='count',x='결혼상태',data=df,order=['divorced','married','etc'],hue='종교유무',palette='Set2');

In [ ]:
# 결혼상태에 따라 종교유무를 나누어 표기 (종교 유 > 무 순으로 표기)
# 테두리 두께를 1로 지정하고, 테두리 색상을 navy로 주기
sns.catplot(kind='count',x='결혼상태',data=df,order=['divorced','married','etc'],hue='종교유무', hue_order=['유','무'],linewidth=1,edgecolor='navy',palette='Set2');

In [ ]:
# 위 그림을 가로막대로 그리고, 막대 색상을 팔레트 "Paired"로 주고, 테두리 색상을 gray로 주기
sns.catplot(kind='count',y='결혼상태',data=df,order=['divorced','married','etc'],hue='종교유무', hue_order=['유','무'],linewidth=1,edgecolor='gray',palette='Paired');

### 연령 분포


In [ ]:
# 연령에 대한 히스토그램
sns.displot(kind='hist',x='연령',data=df);

In [ ]:
# 연령에 대한 히스토그램에 밀도곡선 얹고, 색상을 magenta로 지정하기
sns.displot(kind='hist',x='연령',data=df,binwidth=5,kde=True,color='m');

### 연령과 소득, 결혼유무 분포

In [ ]:
# 연령과 월소득의 산점도
sns.relplot(kind='scatter',x='연령',y='월소득',data=df);

In [ ]:
# 연령과 월소득의 산점도 그린 뒤, 마진에 히스토그램 얹기
sns.jointplot(x='연령',y='월소득',data=df);

In [ ]:
# pairplot으로 그림 그리기
sns.pairplot(data=df[['연령','월소득']])

In [ ]:
# pairplot의 대각선은 밀도곡선으로 변경하고, 하삼각만 그리기
sns.pairplot(data=df[['연령','월소득']],diag_kind='kde',corner=True);

In [ ]:
# 연령과 월소득에 대한 pairplot을 결혼상태에 따라 색상을 구분하여 그리기
sns.pairplot(data=df[['연령','월소득','결혼상태']],hue='결혼상태')

In [ ]:
# 연령과 월소득에 대한 산점도를 결혼상태에 따라 나눠서 그리되, 결혼상태는 결혼과 이혼만 추출해서 그리기
sns.relplot(kind='scatter',x='연령',y='월소득',data=df.query("결혼상태 == 'married' | 결혼상태 == 'divorced'"),col='결혼상태',col_order=['married','divorced'])

In [ ]:
# 연령에 따른 월소득을 회귀도표로 나타내되, 결혼상태에 따라 항목을 나누어서 그리고, 90% 신뢰구간 추가하기
sns.lmplot(x='연령',y='월소득',data=df,col='결혼상태',ci=90)

### 성별과 연령 분포

In [ ]:
# 연령에 대한 박스그림 
sns.catplot(kind='box',x='연령',data=df,width=0.5);

In [ ]:
# 성별에 따라 연령에 대한 바이올린 그림 그리기
sns.catplot(kind='violin',x='성별',y='연령',data=df);

In [ ]:
# 성별에 따른 연령분포에 대한 바이올린 그림을 결혼상태에 따라 나눠 그리기 (단, 결혼상태는 '결혼', '그외'만 추출하여 그리기)
sns.catplot(kind='violin',x='성별',y='연령',data=df.query("결혼상태 != 'divorced'"),hue='결혼상태',hue_order=['married','etc']);

In [ ]:
# 성별에 따른 연령분포에 대한 바이올린 그림을 결혼상태에 따라 분할된 바이올린으로 그리기 (단, 결혼상태는 '결혼', '그외'만 추출하여 그리기)
sns.catplot(kind='violin',x='성별',y='연령',data=df.query("결혼상태 != 'divorced'"),hue='결혼상태',hue_order=['married','etc'],split=True);

### 성별과 소득 분포

In [ ]:
# 성별에 따른 평균 월소득 계산
df.groupby('성별')['월소득'].mean()

In [ ]:
df['월소득'].groupby(df['성별']).mean()

In [ ]:
# 성별에 따른 평균 월소득에 대한 평균막대그래프 (신뢰구간 표시 안하기)
sns.catplot(kind='bar',x='성별',y='월소득',data=df,ci=None);

In [ ]:
# 성별에 따른 평균 월소득에 대한 평균막대그래프를 결혼상태에 따라 구분하여 가로막대로 그리기
sns.catplot(kind='bar',x='성별',y='월소득',data=df,ci=None,hue='결혼상태');

In [ ]:
# 성별에 따른 월소득에 대한 박스그림 그리기
sns.catplot(kind='box',x='성별',y='월소득',data=df)

In [ ]:
# 성별에 따른 월소득에 대한 박스그림 그리기 (평균 월소득이 400 이하인 개체만 추출)
sns.catplot(kind='box',x='성별',y='월소득',data=df.query("월소득 <= 400"))

In [ ]:
# 위 그림에 jitter형태로 데이터 표현하기 (색상은 검정, 점모양은 .으로 표현)
sns.catplot(kind='box',x='성별',y='월소득',data=df.query("월소득 <= 400"))
sns.stripplot(x='성별',y='월소득',data=df.query("월소득 <= 400"),color='k',marker='.')

# 서울시 구별 CCTV 현황 분석

* [서울시 자치구 연도별 CCTV 설치 현황 데이터](http://data.seoul.go.kr/dataList/OA-2734/F/1/datasetView.do)
* [서울시 자치구 연도별 인구 데이터](https://data.seoul.go.kr/dataList/10790/S/2/datasetView.do)
* 서울시 구별 CCTV 설치현황(구별 인구대비현황 등) 분석
* source: [파이썬으로 데이터 주무르기](https://github.com/PinkWink/DataScience)



## 데이터 읽기

In [ ]:
# CCTV 데이터 읽어오기
df = pd.read_csv('/content/drive/MyDrive/colab_data/서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv',encoding='CP949',skiprows=1,thousands=',')
df.head()

In [ ]:
df.info()#data가 문자임.(,때매)

In [ ]:
# 구분이 "계"인 행 삭제
df.shape
df = df[df['구분'] != '계']
df.head()

In [ ]:
# 2021년 열 삭제
df.drop('2021년',axis=1,inplace=True)
df.head()

In [ ]:
#총계 재계산
df['총계'] = df.loc[:,'2012년 이전':'2020년'].sum(axis=1)
df.head()

In [ ]:
# 인구 데이터 읽어오기
pop = pd.read_table(dpath+'report (2).txt',skiprows=2,thousands=',')
pop.head()

In [ ]:
pop = pd.read_table(dpath+'report (2).txt',skiprows=2,thousands=',',usecols=['기간','자치구','계','계.1','계.2','65세이상고령자'])
pop.head()

In [ ]:
pop.rename(columns={'계':'인구수','계.1':'한국인','계.2':'외국인','65세이상고령자':'고령자'},inplace=True)
pop.head()

In [ ]:
# 자치구가 "합계"인 행 삭제
pop = pop[pop['자치구']!='합계']
pop.head()

In [ ]:
# 자치구별 데이터 개수 확인
pop.groupby('자치구')['기간'].count()

In [ ]:
# 연도 최소 최대 확인
pop.describe()

## 데이터 파악



### CCTV 현황

In [ ]:
# CCTV 데이터에서 CCTV 전체개수로 정렬해서 CCTV가 가장 적은 구부터 5개구 확인하기
df.head()
df.sort_values('총계').head()

In [ ]:
# CCTV 데이터에서 CCTV 전체개수로 정렬해서 CCTV가 가장 많은 구부터 5개구 확인하기
df.sort_values('총계',ascending=False).head()

In [ ]:
# 최근 3년간 CCTV 증가율 계산 (최근 3년간 CCTV 수를 더하고, 최근 3년 이전 CCTV 수로 나누기) 
df['최근3년간CCTV설치수'] = df[['2018년','2019년','2020년']].sum(axis=1)
df.head()

In [ ]:
df['최근3년이전CCTV수'] = df.loc[:,'2012년 이전':'2017년'].sum(axis=1)
df['최근3년간CCTV증가율'] = df['최근3년간CCTV설치수'] / df['최근3년이전CCTV수'] * 100
df.head()

In [ ]:
df.drop(['최근3년간CCTV설치수','최근3년이전CCTV수'],axis=1,inplace=True)
df.head()

In [ ]:
# 최근 3년 CCTV 증가율이 가장 높았던 구부터 5개구 확인
df.sort_values("최근3년간CCTV증가율",ascending=False).head()

### 서울시 인구현황

In [ ]:
# 구별 외국인비율과 고령자비율 계산
pop.head()
pop['외국인비율'] = pop['외국인'] / pop['인구수'] * 100
pop['고령자비율'] = pop['고령자'] / pop['인구수'] * 100
pop.head()

In [ ]:
# 2020년 기준 인구수가 가장 많은 구부터 5개구 확인
pop.sort_values('인구수',ascending=False).head()

In [ ]:
# 2020년 기준 외국인 비율이 가장 높은 구부터 5개구 확인
pop.sort_values('외국인비율',ascending=False).head()

In [ ]:
# 2020년 기준 고령자 비율이 가장 높은 구부터 5개구 확인
#pop.loc[pop['기간']==2020].sort_values('고령자비율',ascending=False).head()
pop.sort_values('고령자비율',ascending=False).head()

## 데이터 병합

In [ ]:
# CCTV 데이터에서 불필요한 부분 삭제
df.head()

In [ ]:
x = df.loc[:,'2012년 이전':'2020년'].columns

In [ ]:
df.drop(x,axis=1,inplace=True)
df.head()

In [ ]:
# 서울시 데이터에서 불필요한 부분 삭제 (2020년 기준 자치구, 인구수, 한국인, 외국인, 고령자, 외국인비율, 고령자비율만 남기기)
pop.drop('기간',axis=1,inplace=True)
pop.head()

In [ ]:
# 2020년 데이터만 남기기
#pop.loc[pop['기간'] == 2021,'자치구':]

In [ ]:
# 두 데이터 병합 (구 이름 기준)
final = pd.merge(df,pop,left_on='구분',right_on='자치구')
final.head()

In [ ]:
# 구 이름을 index로 부여하기
final.drop('자치구',axis=1,inplace=True)
final.set_index('구분',inplace=True)
final.head()

## 데이터 분석

### 상관분석


In [ ]:
# CCTV수와 인구수, 고령자비율, 외국인비율 간의 상관관계
# ?pd.DataFrame.corr
final[['총계','인구수','고령자비율','외국인비율']].corr(method='spearman')

In [ ]:
final[['총계','인구수','고령자비율','외국인비율']].corrwith(final['총계'],method='spearman')

In [ ]:
# heatmap으로 나타내기
sns.heatmap(final[['총계','인구수','고령자비율','외국인비율']].corr(method='spearman'),annot=True,fmt=".3f",linewidth=.5,cmap='Blues');

### 구별 CCTV 현황 그래프

In [ ]:
# 구별 CCTV 총계를 가로막대그래프로 그려보기
final.head()

In [ ]:
sns.barplot(x='총계', y=final.index,data=final);

In [ ]:
# 총계가 가장 높은 구부터 차례로 도표
final.sort_values('총계',ascending=False,inplace=True)
final.head()

In [ ]:
sns.barplot(x='총계', y=final.index,data=final);

In [ ]:
# 인구수 대비 CCTV 개수의 비율을 구하고 가장 높은 구부터 도표
final['CCTV비율'] = final['총계'] / final['인구수'] * 100
final.sort_values('CCTV비율',ascending=False,inplace=True)
sns.barplot(x='CCTV비율', y=final.index,data=final);

### 인구수와 CCTV 관계

In [ ]:
# 인구수 대비 CCTV 수에 대한 산점도 그리기
sns.relplot(kind='scatter',x='인구수',y='총계',data=final)
for i in range(final.shape[0]):
  plt.text(x=final['인구수'][i]*1.02,y=final['총계'][i]*0.98,s=final.index[i])

In [ ]:
# 인구수 대비 CCTV 수에 대한 산점도 그리기 (인구수 대비 CCTV가 가장 많은 구부터 5개구만 표기하기)
final.sort_values('CCTV비율',ascending=False,inplace=True)
sns.relplot(kind='scatter',x='인구수',y='총계',data=final)
for i in range(5):
  plt.text(x=final['인구수'][i]*1.02,y=final['총계'][i]*0.98,s=final.index[i])

In [ ]:
# 인구수 대비 CCTV 개수에 대한 회귀도표 그리고 90% 신뢰구간 표기하기
sns.regplot(x='인구수',y='총계',data=final,ci=90);

# 일자리와 부동산 관계 분석

* [지역별 고용노동통계 데이터](http://laborstat.moel.go.kr/hmp/index.do)
* [KOSIS 주민등록세대수](https://kosis.kr/)
* [KOSIS 아파트 실거래 가격 지수](https://kosis.kr/)

* source: [파이썬으로 데이터 주무르기](https://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791158391546)


## 데이터 읽기


In [ ]:
# 지역별 일자리 데이터 읽어오기
job = pd.read_csv('/content/drive/MyDrive/colab_data/행정구역_시도__산업별_대분류__고용_20211209021823.csv',encoding='CP949',skiprows=1)
job

In [ ]:
# 지역명 나누기
def get_sido(x):
  if len(x) == 4:
    return x[0] + x[2]
  else:
    return x[:2]

job['지역'] = job['지역별'].apply(get_sido)
job.head()

In [ ]:
job.rename(columns={job.columns[2]:'종사자수'},inplace=True)
job.head()

In [ ]:
job.drop(job.columns[:2],axis=1,inplace=True)
job.head()

In [ ]:
# 세대수 데이터
pop = pd.read_csv('/content/drive/MyDrive/colab_data/행정구역_시군구_별_주민등록세대수_20211209022025.csv',encoding='CP949')
pop.head()

In [ ]:
pop['지역'] = pop.iloc[:,0].apply(get_sido)
pop.head()

In [ ]:
pop.drop(pop.columns[0],axis=1,inplace=True)
pop.head()

In [ ]:
pop.rename(columns={'2020':'세대수'},inplace=True)
pop.head()

In [ ]:
## 아파트 매매가격지수
apart = pd.read_csv('/content/drive/MyDrive/colab_data/아파트_매매_실거래가격지수_20211209023155.csv',encoding='CP949',skiprows=1)
apart.drop(apart.columns[0],axis=1,inplace=True)
apart.head()

In [ ]:
apart.rename(columns={apart.columns[0]:'지역'},inplace=True)
apart.head()

In [ ]:
apart['아파트매매가지수'] = apart.iloc[:,1:].mean(axis=1)
apart.head()

In [ ]:
apart = apart[['지역','아파트매매가지수']]
apart.head()

## 데이터 병합

In [ ]:
temp = pd.merge(job,pop,on='지역')
temp.head()

In [ ]:
df = pd.merge(temp,apart,on='지역')
df.head()

In [ ]:
df = df[['지역','세대수','종사자수','아파트매매가지수']]
df.head()

## 데이터 분석


In [ ]:
# 시도별 고용자수를 막대그래프로 그리기
sns.barplot(x='지역',y='종사자수',data=df);

In [ ]:
# 고용자수 높은 시도부터 도표
df.sort_values('종사자수',ascending=False,inplace=True)
sns.barplot(x='지역',y='종사자수',data=df);

In [ ]:
# 고용자수 높은 시도부터 도표 (평균 고용자수 라인 추가)
df.sort_values('종사자수',ascending=False,inplace=True)
sns.barplot(x='지역',y='종사자수',data=df)
plt.axhline(y=df['종사자수'].mean(),c='orange',lw=2,ls='--');

In [ ]:
# 세대수 대비 고용비율 컬럼 추가
df['세대수대비종사자비율'] = df['종사자수'] / df['세대수']
df.sort_values('세대수대비종사자비율',ascending=False,inplace=True)
df.head()

In [ ]:
# 시도 단위 세대수 대비 고용비율 도표
sns.barplot(x='지역',y='세대수대비종사자비율',data=df,color='seagreen')
plt.axhline(y=df['세대수대비종사자비율'].mean(),c='red',lw=1.5,ls=':');

In [ ]:
# 세대수 대비 고용률과 아파트 매매가지수 회귀도표
sns.lmplot(x='세대수대비종사자비율',y='아파트매매가지수',data=df);
for i in range(df.shape[0]):
  plt.text(x=df['세대수대비종사자비율'][i],y=df['아파트매매가지수'][i],s=df['지역'][i]);

In [ ]:
# 시도별 고용자수, 세대수를 아파트 매매가지수와 함께 살펴보기
sns.relplot(kind='scatter',data=df,x='종사자수',y='세대수',hue='아파트매매가지수',size='아파트매매가지수',sizes=(40,500));

In [ ]:
# 시도별 세대수 대비 고용률, 세대수를 아파트 매매가지수와 함께 살펴보기 ()
sns.relplot(kind='scatter',data=df,x='세대수대비종사자비율',y='세대수',hue='아파트매매가지수',size='아파트매매가지수',sizes=(40,500));
for i in range(df.shape[0]):
  plt.text(x=df['세대수대비종사자비율'][i],y=df['세대수'][i],s=df['지역'][i]);

# THE END